In [4]:
import environ
env = environ.Env()
environ.Env.read_env('.env')

In [5]:
HUGGING_TOKEN=env("HUGGING_TOKEN")

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_community.chat_models import ChatOllama
from langchain import HuggingFaceHub, HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from huggingface_hub import login
import PyPDF2

/Users/hamdaniilham/Python/AI-MCQGen/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#login(token=HUGGING_TOKEN)

In [3]:
# Set up the OpenAI API client
#llm=ChatOpenAI(openai_api_key=KEY, temperature=0, model_name="gpt-3.5-turbo")

# Set up the Ollama
llama=ChatOllama(model='phi3')

#set up the hugging face model
# model_id = "google/flan-t5-large"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, low_cpu_mem_usage=True,)
# pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)
# local_llm = HuggingFacePipeline(pipeline=pipeline)

In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "soal pilihan ganda",
        "options": {
            "a": "pilihannya disini",
            "b": "pilihannya disini",
            "c": "pilihannya disini",
            "d": "pilihannya disini",
        },
        "correct": "correct answer",
    },
    "2": {
          "mcq": "soal pilihan ganda",
        "options": {
            "a": "pilihannya disini",
            "b": "pilihannya disini",
            "c": "pilihannya disini",
            "d": "pilihannya disini",
        },
        "correct": "correct answer",
    },
    "3": {
          "mcq": "soal pilihan ganda",
        "options": {
            "a": "pilihannya disini",
            "b": "pilihannya disini",
            "c": "pilihannya disini",
            "d": "pilihannya disini",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE="""
Text={text}
Nama kamu adalah DhankMaker, Kamu adalah seorang instruktur atau guru yang sangat handal dalam segala bidang, 
yang ditugaskan untuk membuat soal dan jawaban pilihan ganda menggunakan bahasa indonesia.
buat soal pilihan ganda yang jumlah soalnya {number} dengan tema {subject} dan diperuntukkan bagi siswa {level} dengan tingkat kesulitan {tone} .
pastikan bahwa soalnya tidak berulang dan pastikan semua soalnya dalam bentuk teks
pastikan format respon nya seperti RESPONSE_JSON dibawah dan gunakan itu sebagai petunjuk 
pastikan untuk membuat soal {number}
### RESPONSE_JSON
{response_json}
"""

In [7]:
quiz_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json", "level"],
    template=TEMPLATE,
)

In [25]:
llm_hugging=HuggingFaceHub(repo_id='openai-community/gpt2', model_kwargs={'temperature':0.1}, huggingfacehub_api_token=HUGGING_TOKEN)

In [26]:
quiz_chain=LLMChain(llm=llm_hugging, prompt=quiz_prompt, output_key="quiz", verbose=True)

In [20]:
TEMPLATE2="""
Kamu adalah seorang ahli tata bahasa indonesia dan penulis bahasa indonesia. Diberikan Kuis Pilihan Ganda untuk siswa {level}.
Anda perlu mengevaluasi kompleksitas pertanyaan dan memberikan analisis kuis secara lengkap. Hanya gunakan maksimal 50 kata untuk analisis kompleksitas. 
jika kuis tidak sesuai dengan kemampuan kognitif dan analitis siswa,
perbarui soal kuis yang perlu diubah dan ubah nadanya agar sesuai dengan kemampuan siswa
Kuis_MCQ:
{quiz}

Periksa dari Penulis ahli Bahasa Indoesia dari kuis di atas:
"""

In [27]:
quiz_eval=PromptTemplate(
    input_variables=["quiz","level"],
    template=TEMPLATE2
)

In [28]:
review_chain=LLMChain(llm=llm_hugging, prompt=quiz_eval, output_key="eval", verbose=True)

In [23]:
generate_eval=SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json", "level"],
    output_variables=["quiz", "eval"],
    verbose=True
)

In [29]:
# with get_openai_callback() as cb:
#     response=generate_eval({
#         "text": "Soal Pilihan Ganda",
#         "number": 10,
#         "subject": "Kecepatan dan Jarak",
#         "tone": "Mudah",
#         "response_json": RESPONSE_JSON,
#         "level": "SMP"
#     })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text=Soal Pilihan Ganda
Nama kamu adalah DhankMaker, Kamu adalah seorang instruktur atau guru yang sangat handal dalam segala bidang, 
yang ditugaskan untuk membuat soal dan jawaban pilihan ganda menggunakan bahasa indonesia.
buat soal pilihan ganda yang jumlah soalnya 10 dengan tema Kecepatan dan Jarak dan diperuntukkan bagi siswa SMP dengan tingkat kesulitan Mudah .
pastikan bahwa soalnya tidak berulang dan pastikan semua soalnya dalam bentuk teks
pastikan format respon nya seperti RESPONSE_JSON dibawah dan gunakan itu sebagai petunjuk 
pastikan untuk membuat soal 10
### RESPONSE_JSON
{'1': {'mcq': 'soal pilihan ganda', 'options': {'a': 'pilihannya disini', 'b': 'pilihannya disini', 'c': 'pilihannya disini', 'd': 'pilihannya disini'}, 'correct': 'correct answer'}, '2': {'mcq': 'soal pilihan ganda', 'options': {'a': 'pilihannya disini', 'b': 'pilihannya disini', 'c': 'pilihannya disi

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/openai-community/gpt2 (Request ID: UUsG8xZaVzRY19Jj0pAPc)

Input validation error: `temperature` must be strictly positive
Make sure 'text-generation' task is supported by the model.

In [30]:
response=generate_eval({
    "text": "Soal Pilihan Ganda",
    "number": 10,
    "subject": "Kecepatan dan Jarak",
    "tone": "Mudah",
    "response_json": RESPONSE_JSON,
    "level": "SMP"
})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text=Soal Pilihan Ganda
Nama kamu adalah DhankMaker, Kamu adalah seorang instruktur atau guru yang sangat handal dalam segala bidang, 
yang ditugaskan untuk membuat soal dan jawaban pilihan ganda menggunakan bahasa indonesia.
buat soal pilihan ganda yang jumlah soalnya 10 dengan tema Kecepatan dan Jarak dan diperuntukkan bagi siswa SMP dengan tingkat kesulitan Mudah .
pastikan bahwa soalnya tidak berulang dan pastikan semua soalnya dalam bentuk teks
pastikan format respon nya seperti RESPONSE_JSON dibawah dan gunakan itu sebagai petunjuk 
pastikan untuk membuat soal 10
### RESPONSE_JSON
{'1': {'mcq': 'soal pilihan ganda', 'options': {'a': 'pilihannya disini', 'b': 'pilihannya disini', 'c': 'pilihannya disini', 'd': 'pilihannya disini'}, 'correct': 'correct answer'}, '2': {'mcq': 'soal pilihan ganda', 'options': {'a': 'pilihannya disini', 'b': 'pilihannya disini', 'c': 'pilihannya disi

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/openai-community/gpt2 (Request ID: nRCycmFlo2cGEYTyE5-_J)

Input validation error: `temperature` must be strictly positive
Make sure 'text-generation' task is supported by the model.

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0
